In [15]:
from tools.token.s3 import upload_to_s3
import json
import requests
import os
from datetime import datetime
import uuid
import pytz

In [16]:
cookies = {
    'id': 'ed0e34f9aae74b2cbffd67b776a00ad5:1rOCNJ:XnTbL44YAIqn2N6Ut01BhF1q0_c',
    'csrftoken': 'ZuewSrCtAqARq0UTfg3zDxDbu4bPwRaf0hrb10YNlASaYKPc1hlQV9EOKkPlIx04',
    '_ga': 'GA1.1.148563303.1705046085',
    '_fbp': 'fb.1.1705046086133.2069485138',
    '_ga_EPNV57ZWVB': 'GS1.1.1706259932.1.1.1706260143.0.0.0',
    '_ga_RQ89S82FH7': 'GS1.1.1706261041.82.1.1706261141.60.0.0',
}

headers = {
    'authority': 'www.inhersight.com',
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.9,id;q=0.8',
    # 'cookie': 'id=ed0e34f9aae74b2cbffd67b776a00ad5:1rOCNJ:XnTbL44YAIqn2N6Ut01BhF1q0_c; csrftoken=ZuewSrCtAqARq0UTfg3zDxDbu4bPwRaf0hrb10YNlASaYKPc1hlQV9EOKkPlIx04; _ga=GA1.1.148563303.1705046085; _fbp=fb.1.1705046086133.2069485138; _ga_EPNV57ZWVB=GS1.1.1706259932.1.1.1706260143.0.0.0; _ga_RQ89S82FH7=GS1.1.1706261041.82.1.1706261141.60.0.0',
    'if-none-match': 'W/"2392a51ddf77ec5a56a9fd9fba3f75c2"',
    'referer': 'https://www.inhersight.com/companies',
    'sec-ch-ua': '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}

params = {
    'for_discovery': 'true',
    'page_size': '100',
    'order_by': 'priority:desc',
    'is_paying': 'true',
}

response = requests.get('https://www.inhersight.com/api/v1/companies', params=params, cookies=cookies, headers=headers)

In [17]:
result_json = response.json()['results']

In [ ]:
#logging monitor dan logging monitor error

def save_to_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=2)

def data_id_generate():
    return int(uuid.uuid4().hex[:16], 16)

def main():
    monitor_data = []
    monitor_data_log=[]
    total_success = 0
    total_failed = 0

    def monitor_data_log_error(error_type='', message=''):
        try:
            monitor_data_log.append({
                'Crawlling_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'id_project': None,
                'project': 'Data Intelligence',
                'sub_project': 'data review',
                'source_name': 'inhersight.com',
                'sub_source_name': item['name'],
                'id_sub_source': item['id'],
                'id_data': data_id_generate(),
                'process_name': 'Crawling',
                'status': 'Success' if error_type == '' else 'Error',  
                'type_error': error_type, 
                'message': message, 
                'assign': 'Daud'
            })
            save_to_json(monitor_data_log, 'monitoring_log_error.json')
        except Exception as e:
            print(f'Error in monitor_data_log_error: {str(e)}')


    def monitor_data_json():
        monitor_data.append({
            'Crawlling_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'id_project': None,
            'project': 'Data Intelligence',
            'sub_project': 'data review',
            'source_name': 'inhersight.com',
            'sub_source_name': item['name'],
            'id_sub_source': item['id'],
            'total_data': total_success + total_failed,
            'total_success': total_success,
            'total_failed': total_failed,
            'status': 'Done',
            'assign': 'Daud'
        })
        save_to_json(monitor_data, 'monitoring_data.json')

    if response.status_code == 200:
        result_json = response.json()['results']

        name_path_folder_raw = 'data_inhersight'

        for item in result_json:

            try:

                monitor_data_log_error()
            
                company_name = item['name']
                clean_name = f'{company_name.replace(' ', '_')}.json'
                filename = f'{clean_name}_{item['id']}_---.json'
                dir_path_raw = os.path.join(name_path_folder_raw)
                os.makedirs(dir_path_raw, exist_ok=True)
                #! your local path
                local_path = f'D:/Visual Studio Code/Project Done/Magang Github/API/Inhersight/data_inhersight/{filename}'
                path_s3 = f's3://ai-pipeline-statistics/data/data_raw/data_review/inhersight/companies/{filename}'
                company_data = {
                    'link:': item['url'],
                    'domain': 'inhersight',
                    'tag': [
                        'https://www.inhersight.com',
                        'companies',
                        item['name']
                    ],
                    'crawling_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'crawling_time_epoch': int(datetime.now().timestamp()),
                    'path_data_raw': path_s3,
                    'path_data_clean': None,
                    'reviews_name': item['name'],
                    'category_reviews': 'Kerja',
                    'id': item['id'],
                    'industry': item['industry'],
                    'total_jobs': item['num_jobs'],
                    'size': item['size'],
                    'logo_url': item['logo_square_url'],
                    'employee_count': item['employee_count'],
                    'desc': item['short_description'],
                    'total_score_count': item['score_count'],
                    'total_reviews': None,
                    'detail_total_rating': {
                        'reviews_rating': float(item['rounded_weighted_composite_score']) if item['rounded_weighted_composite_score'] else None,
                        'detail_total_rating': [{
                        }],
                    }
                }

                company_id = item['id']
                page_number = 1
                page_size = 100
                target_comment_count = 300
                total_comments = []

                while len(total_comments) < target_comment_count:
                    comments_url = f'https://www.inhersight.com/api/v1/companies/{company_id}/comments?page={page_number}&page_size={page_size}'
                    comments_response = requests.get(comments_url, cookies=cookies, headers=headers)



                    if comments_response.status_code == 200:
                        comments_data = comments_response.json()['results']
                        comment_count = comments_response.json()['count']
                        total_comments.extend(comments_data)
                        len_comment = len(total_comments)
                        for created in comments_data:
                            desired_timezone = pytz.timezone('UTC')
                            created_obj = created['created']
                            parsed_datetime = datetime.strptime(created_obj, '%Y-%m-%dT%H:%M:%S.%f%z').replace(tzinfo=pytz.utc).astimezone(desired_timezone)
                            formatted_datetime = parsed_datetime.strftime('%Y-%m-%d %H:%M:%S')
                            epoch_timestamp = int(parsed_datetime.timestamp())

                            
                        try:
                            
                            for i, comment in enumerate(comments_data):
                                clean_name = f'{company_name.replace(' ', '_')}.json'
                                comment_filename = f'{clean_name}_{item['id']}_{comment['id']}.json'
                                #! your local path
                                local_path_comment = f'D:/Visual Studio Code/Project Done/Magang Github/API/Inhersight/data_inhersight/{comment_filename}'
                                path_s3_comment = f's3://ai-pipeline-statistics/data/data_raw/data_review/inhersight/companies/{comment_filename}'
                                data_comment = {
                                    'link': item['url'],
                                    'domain': 'inhersight',
                                    'tag': [
                                        'https://www.inhersight.com',
                                        'companies',
                                        item['name']
                                    ],
                                    'crawling_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                                    'crawling_time_epoch': int(datetime.now().timestamp()),
                                    'path_data_raw': path_s3_comment,
                                    'path_data_clean': None,
                                    'reviews_name': item['name'],
                                    'category_reviews': 'Kerja',
                                    'id': item['id'],
                                    'industry': item['industry'],
                                    'total_jobs': item['num_jobs'],
                                    'size': item['size'],
                                    'logo_url': item['logo_square_url'],
                                    'employee_count': item['employee_count'],
                                    'desc': item['short_description'],
                                    'total_score_count': item['score_count'],
                                    'total_reviews': int(comment_count),
                                    'reviews_rating': {
                                        'total_rating': float(item['rounded_weighted_composite_score']) if item['rounded_weighted_composite_score'] else None,
                                        'detail_total_rating': [{
                                        }]
                                    },
                                    'detail_reviews' : {
                                        'username_reviews' : comment['satisfaction_slug'],
                                        'image_reviews' : comment['avatar'],
                                        'created_time': formatted_datetime,
                                        'created_time_epoch' : epoch_timestamp,
                                        'company_name' : item['name'],
                                        'id_reviews': comment['id'],
                                        'job_level' : comment['job_level'],
                                        'job_level_display': comment['job_level_display'],
                                        'title_detail_reviews' : comment['satisfaction_display'],
                                        'reviews_rating' : comment['satisfaction_score'],
                                        'detail_review_rating':[],
                                        'content_reviews': comment['comment_text'],
                                        'total_reply_reviews': len(comment['replies']),
                                        'reply_content_reviews': comment['replies'],

                                        'date_of_experience' : formatted_datetime,
                                        'date_of_experience_epoch' : epoch_timestamp
                                    }
                                }

                                comment_path_raw = os.path.join(dir_path_raw, comment_filename)
                                save_to_json(data_comment, comment_path_raw)
                                # upload_to_s3(local_path_comment, path_s3_comment.replace('s3://', ''))


                                total_success += 1
                        except:
                            total_failed += 1

                        print('Success get data from: ' + f'{item["name"]}')

                        if len(comments_data) >= page_size:
                            page_number += 1
                        else:
                            break 

                        if 'next' in comments_response.json() and comments_response.json()['next']:
                            comments_url = comments_response.json()['next']
                        else:
                            break 
                    else:
                        print(f'Failed to retrieve comments for company {item["name"]}. Status code: {comments_response.status_code}')
                        break
                        
                monitor_data_json()
                total_success = 0
                total_failed = 0


                if len_comment == 0:
                    path_data_raw = os.path.join(dir_path_raw, filename)
                    save_to_json(company_data, path_data_raw)
                    # upload_to_s3(local_path, path_s3.replace('s3://', ''))

                else:
                    pass

            except KeyboardInterrupt:
                print('Process interrupted by user.')
                monitor_data_log_error('KeyboardInterrupt', 'Process interrupted by user.')
                break

            except requests.Timeout:
                print('Request timed out.')
                monitor_data_log_error('TimeoutError', 'Request timed out.')


            except Exception as e:
                print(f'An unexpected error occurred: {str(e)}')
                monitor_data_log_error('UnexpectedError', f'An unexpected error occurred: {str(e)}')
                
                

    else:
        print('Error:', response.status_code)
        print('Gagal mengambil data dari API')
    print('Proccess Completed')

main()